Super Store Sales Use-Case Data Analytics and Visualization

In Phase 1 of the final project, I started by exploring multiple datasets from sources like data.world, data.illinois.gov, data.gov,developer.marvel.com, IDB, kaggle.com,uci machine learning repository and a few other resources. After indepth analysis, I finalised Global Superstore dataset as it gave me more flexibility to play around with its fields. Going ahead in Phase 2 I was able to add user interactivity and make intuitive dashboards by using the various columns in my Global Superstore dataset.
You can obtain this dataset from data.world by clicking on this url https://data.world/2918diy/global-superstore/workspace/file?filename=Global+Superstore.txt
You will have to first login to data.world to open the above link.

In [1]:
#importing these libraries needed to carry out visualizations and interactive dashboards from my dataset.
import pandas as pd
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
import bqplot
import bqplot.pyplot

In [2]:
#Importing the data(Below is the relative path of the data)
global_superstore= pd.read_csv('Global Superstore.txt',delimiter= "\t")

In [3]:
pivot_table=pd.pivot_table(global_superstore, index="Sub-Category", columns="Market",values="Profit",aggfunc='mean',dropna=True,fill_value=0)

In [4]:
#Viewing my pivot table
pivot_table

Market,APAC,Africa,Canada,EMEA,EU,LATAM,US
Sub-Category,,,,,,,
Accessories,22.722624,27.923198,68.174211,14.930763,73.985715,41.045802,54.111788
Appliances,124.641151,25.138808,139.623750,19.636675,148.517413,80.954019,38.922758
Art,10.986138,6.425288,14.731452,2.086754,21.473992,11.432066,8.200737
Binders,11.189254,4.344402,12.680323,4.186675,15.003964,5.435112,19.843574
Bookcases,108.423892,36.555704,103.271538,35.757865,116.543944,38.645193,-15.230509
Chairs,64.059823,15.213574,71.395000,-2.438280,42.562510,30.749137,43.095894
Copiers,124.009272,68.003068,204.895385,43.043732,120.859471,65.255076,817.909190
Envelopes,8.384861,7.403078,17.118000,3.510117,19.592905,9.348811,27.418019
Fasteners,2.533819,3.847284,7.785000,4.038846,10.225406,3.993060,4.375660


In [5]:
# 1. Scales --for Bar Plot
# I have referred to my Assignment 6.
x_sch = bqplot.OrdinalScale()
y_sch = bqplot.LinearScale()

# 2. Axis
x_axh = bqplot.Axis(scale=x_sch,label="Year")
y_axh = bqplot.Axis(scale=y_sch, orientation="vertical",label="Total Profit")

# 3. Marks -- using bar plot
bar_to_plot=bqplot.pyplot.bar(x=[], y=[], scales={'x':x_sch, 'y':y_sch})

# 4. figure!
fig_barplot = bqplot.Figure(marks=[bar_to_plot], axes=[x_axh, y_axh])
#fig_barplot

In [6]:
# step1 to add scales - colors, x & y
# Referred to Jill Naiman's class code and my assignment 6(homework)
col_sc = bqplot.ColorScale(scheme="Orange") 
x_sc = bqplot.OrdinalScale() 
y_sc = bqplot.OrdinalScale()

#step2. Axis -- color & for x/y
ax_col = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
ax_x = bqplot.Axis(scale=x_sc,label="Market")
ax_y = bqplot.Axis(scale=y_sc, orientation='vertical',label="Sub-Category")

# Step3. Using heatmap to plot
heat_map = bqplot.GridHeatMap(color=pivot_table.values,row =pivot_table.index,column =pivot_table.columns,
                             scales={'color':col_sc, 'row':y_sc, 'column':x_sc}, interactions={'click':'select'},
                        anchor_style={'fill':'blue'},unselected_style = {'opacity': 2.0})

#step4 Defined a function for the bar plot
def in_barplot(a,b):
    val=global_superstore[(global_superstore["Sub-Category"]==a) & (global_superstore["Market"]==b)]
    val = val[val['Year'] != 0] 
    val=val.groupby("Year")["Profit"].sum()
    return val

# step5. building interactions
mySelectedLabel = ipywidgets.Label()
mySelectedLabel.value = 'Mean selected point here'
# step6 interaction function -- this is what happens on selection:
def on_selected(change):
    #print(change)
    if len(change['owner'].selected) == 1: # this will allow the user to select one bin
        i,j = change['owner'].selected[0] # the only selected bin
        v = pivot_table.values[i,j] #passing pivot1.values
        mySelectedLabel.value = 'Mean Profit for Sub-Category: ' + pivot_table.index[i] + ' in Market: ' + pivot_table.columns[j] + ' = ' + str(v)
# step 7 for linking  interaction function to the mark:
        new_superstore=in_barplot(pivot_table.index[i],pivot_table.columns[j])
    
        bar_to_plot.x=new_superstore.index
        bar_to_plot.y=new_superstore.values
heat_map.observe(on_selected, 'selected')

# step8 Finally, putting together in a figure
fig_heatmap = bqplot.Figure(marks=[heat_map], axes=[ax_col, ax_x, ax_y])
fig_barplot = bqplot.Figure(marks=[bar_to_plot], axes=[x_axh, y_axh])

# step9 to display on a dashboard.
myDashboard = ipywidgets.VBox([mySelectedLabel, ipywidgets.VBox([fig_heatmap,fig_barplot])])
myDashboard

This is an interactive dashboard. For the purpose of this dashboard I have used two visualizations mainly a Bar graph and a Grid Heat map. Starting with the Grid Heat map, which is used to depict values for a main variable of interest across two axis variables as a grid of colored squares.This main variable of interest is the mean of profit values for sub-category products in each of the market regions. This simply means that a user working on my dashboard will be able to visualize through this grid heat map what is the profit achieved by the sale of each sub category products like Chairs, Bookcases, Blinders, Appliances, Accessories, Labels, Phones, Tables etc in various markets across the globe.

When a user wants to find the profit made by a particular sub-category in a specific market then the user will select a particular cell of the grid heat map. For instance the user wants to find out in Canada what was the profit made by chairs then by selecting the quivalent cell the mean value of the profit made in Canada can be obtained. This can then be compared with the profit values across all the sub-categories in Canada or using a particular sub-category(chairs) to compare it's mean profit value across different markets globally.

Next to add another dimension to my visualization I have included a bar graph. This bar chart shows the total profit in each year when a cell from the heat map is selected i.e a product sub-category in a  particular market  is selected.

My dashboard is interactive. By selecting a cell one will get the mean value of profit of sub-category beloging to a particular market. Furthermore, the bar chart then breaks the profit down to all years and shows the total profit of the product sub-category made in each of the year in that market. For instance in the above example, when the cell is selected for the mean of profit for chairs in Canada then the bar graph will display in each year what was the total profit made by chairs in Canada.

In [7]:
pivot_table1 =pd.pivot_table(global_superstore, index="Sub-Category", columns="Order Priority",values="Sales",aggfunc='mean',dropna=True,fill_value=0)

In [8]:
#Viewing my pivot table
pivot_table1

Order Priority,Critical,High,Low,Medium
Sub-Category,,,,
Accessories,254.644068,248.732582,225.286765,240.769543
Appliances,597.548780,582.164510,588.528571,568.301020
Art,76.528926,76.215793,75.034335,76.273598
Binders,66.577253,65.086197,93.240000,79.834506
Bookcases,493.994975,602.284314,575.496183,631.187271
Chairs,479.439834,442.905181,364.747126,435.660822
Copiers,717.502959,663.258544,546.530973,694.045741
Envelopes,72.512563,69.203274,67.518182,70.597990
Fasteners,34.543956,35.799724,33.236641,33.762834


In [9]:
# 1. Scales --for Bar Plot
# I have referred to my Assignment 6.
x_sch1 = bqplot.OrdinalScale()
y_sch1 = bqplot.LinearScale()

# 2. Axis
x_axh1 = bqplot.Axis(scale=x_sch1,label="Ship Mode")
y_axh1 = bqplot.Axis(scale=y_sch1, orientation="vertical",label="Total Sales")

# 3. Marks -- using bar plot
bar_to_plot1=bqplot.pyplot.bar(x=[], y=[], scales={'x':x_sch1, 'y':y_sch1})

# 4. figure!
fig_barplot1 = bqplot.Figure(marks=[bar_to_plot1], axes=[x_axh1, y_axh1])
#fig_barplot

In [19]:
# step1 to add scales - colors, x & y
# Referred to Jill Naiman's class code and my assignment 6(homework)
col_sc1 = bqplot.ColorScale(scheme="Purple") 
x_sc1 = bqplot.OrdinalScale() 
y_sc1 = bqplot.OrdinalScale()

#step2. Axis -- color & for x/y
ax_col1 = bqplot.ColorAxis(scale=col_sc1, orientation='vertical', side='right')
ax_x1 = bqplot.Axis(scale=x_sc1,label="Order Priority")
ax_y1 = bqplot.Axis(scale=y_sc1, orientation='vertical',label="Sub-Category")

# Step3. Using heatmap to plot
heat_map1 = bqplot.GridHeatMap(color=pivot_table1.values,row =pivot_table1.index,column =pivot_table1.columns,
                             scales={'color':col_sc1, 'row':y_sc1, 'column':x_sc1}, interactions={'click':'select'},
                        anchor_style={'fill':'blue'},unselected_style = {'opacity': 2.0})

#step4 Defined a function for the bar plot
def in_barplot1(a,b):
    val=global_superstore[(global_superstore["Sub-Category"]==a) & (global_superstore["Order Priority"]==b)]
    val = val[val['Ship Mode'] != 0] 
    val=val.groupby("Ship Mode")["Sales"].sum()
    return val

# step5. building interactions
mySelectedLabel1 = ipywidgets.Label()
mySelectedLabel1.value = 'Mean selected point here'
# step6 interaction function -- this is what happens on selection:
def on_selected1(change):
    #print(change)
    if len(change['owner'].selected) == 1: # this will allow the user to select one bin
        i,j = change['owner'].selected[0] # the only selected bin
        v = pivot_table1.values[i,j] #passing pivot1.values
        mySelectedLabel1.value = 'Mean Sales for Sub-Category: ' + pivot_table1.index[i] + ' in Order Priority: ' + pivot_table1.columns[j] + ' = ' + str(v)
# step 7 for linking  interaction function to the mark:
        new_superstore=in_barplot1(pivot_table1.index[i],pivot_table1.columns[j])
    
        bar_to_plot1.x=new_superstore.index
        bar_to_plot1.y=new_superstore.values
heat_map1.observe(on_selected1, 'selected')

# step8 Finally, putting together in a figure
fig_heatmap1 = bqplot.Figure(marks=[heat_map1], axes=[ax_col1, ax_x1, ax_y1])
fig_barplot1 = bqplot.Figure(marks=[bar_to_plot1], axes=[x_axh1, y_axh1])

# step9 to display on a dashboard.
myDashboard1 = ipywidgets.VBox([mySelectedLabel1, ipywidgets.VBox([fig_heatmap1,fig_barplot1])])
myDashboard1

This is another interactive dashboard. Here I am using a grid heat map to display the mean sales of each of the sub-categories based on the order priority which could be critical, high, low and medium. Here one can see that the maximum mean sales are for tables sub-category based on the order priority. To get useful insights I have linked it with a bar graph. This bar graph depicts the total sales based on the shipping mode.

By selecting a cell on the heat map one will get the mean sales of a specific sub-category based on the order category. Additionally the bar graph will break this information to get the total sales based on the shipping mode. 

Logically any sub-category product which is sold on a low order priority will opt for a standard shipping and that is exactly what my dashboard is depicting. When one selects any of the sub-category products sold on the low order priority then the bar graph will show only a standard class shipping mode.
Similarly when one selects any of the sub-category products sold on a critical order priority then the bar graph will show only 3 shipping modes which are Same day shipping mode, First class shipping mode or Second class shipping mode. However, none of the critical order priority sub-categories was sold with a standard shipping mode.

I identified a contextual dataset which is similar to my original global superstore dataset. This is a coffee chain dataset as is useful for my data story telling with respect to the original global superstore data. I got coffee chain dataset from data.world.The link to this is https://data.world/2918diy/coffee-chain/workspace/file?filename=Coffee+Chain.txt

This dataset like my original dataset(global superstore) has similar fields like the location fields(State,Market) which helps us identify which types of coffee or coffee products are present or sold in different locations. Here I also have fields like Sales and Profit which will help me identify the sales and profit regionwise or statewise or productwise as we have fields like Product, Product Type, Product Line. These fields are similar to the Global Superstore Dataset which I have chosen in my Part 1 and Part 2 of my project. As a result I can make similar visualizing plots and similar interactive dashboards in this new dataset i.e the coffee chain dataset as I have made in global superstore dataset.

However, this coffee chain dataset has 2 new important fields like the Target Profit and Taget Sales. This helps me find the comparison between Sales and Target Sales or Profit and Taget Profit. Here I can visualise values of actual Sales with Target Sales and draw a comparison to see how the targeted sales have changed(increased or decreased) with respect to actual Sales. Similarly I can do that with Profit and Target Profit fields from this coffee chain data.

Going ahead I will be an interactive visualization and a dashboard which will help provide data insights from this dataset.

In [11]:
#This is the coffee chain dataset.
global_coffee= pd.read_csv('Coffee Chain.txt',delimiter= "\t")

In [12]:
global_coffee

,Area Code,Cogs,Difference Between Actual and Target Profit,Date,Inventory,Margin,Market Size,Market,Marketing,记录数,...,Product,Profit,Sales,State,Target COGS,Target Margin,Target Profit,Target Sales,Total Expenses,Type
0,303,51,-35,2012-10-01 00:00:00.000,503,71,Major Market,Central,46,1,...,Lemon,-5,122,Colorado,30,60,30,90,76,Decaf
1,970,52,-24,2012-10-01 00:00:00.000,405,71,Major Market,Central,17,1,...,Mint,26,123,Colorado,30,60,50,90,45,Decaf
2,409,43,-22,2012-10-01 00:00:00.000,419,64,Major Market,South,13,1,...,Lemon,28,107,Texas,30,60,50,90,36,Decaf
3,850,38,-15,2012-10-01 00:00:00.000,871,56,Major Market,East,10,1,...,Darjeeling,35,94,Florida,40,60,50,100,21,Regular
4,562,72,6,2012-10-01 00:00:00.000,650,110,Major Market,West,23,1,...,Green Tea,56,182,California,20,60,50,80,54,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,775,250,133,2013-12-01 00:00:00.000,1820,251,Small Market,West,70,1,...,Earl Grey,233,534,Nevada,180,180,100,360,94,Regular
1058,971,88,48,2013-12-01 00:00:00.000,817,133,Small Market,West,29,1,...,Earl Grey,108,236,Oregon,60,100,60,160,60,Regular
1059,775,294,-285,2013-12-01 00:00:00.000,8252,-294,Small Market,West,111,1,...,Green Tea,-605,33,Nevada,210,-210,-320,0,145,Regular
1060,503,134,80,2013-12-01 00:00:00.000,690,186,Small Market,West,41,1,...,Green Tea,180,341,Oregon,90,140,100,230,65,Regular


In [13]:
field_bin= ["State","Market","Product Type","Product Line","Product"]
value_bin=["Sales","Profit"]

In [14]:
@ipywidgets.interact(Field = field_bin,values_choice=value_bin,style=plt.style.available)
def make_plot(style,Field,values_choice):
    with plt.style.context(style): 
        fig, ax = plt.subplots(1,1,figsize=(15,3))
        if values_choice in ["Sales"]:
            df=global_coffee.groupby(Field)["Sales"].sum()
            df2=global_coffee.groupby(Field)["Target Sales"].sum()
            ax.bar(df.index,df.values)
            ax.plot(df2.index,df2.values, color="red", marker=".")
            ax.set_xlabel(Field)
            ax.set_ylabel("Actual Sales compared to Target Sales")
            ax.legend(labels=['Target Sales', "Sales"])
        
        else:
            df3=global_coffee.groupby(Field)["Profit"].sum()
            df4=global_coffee.groupby(Field)["Target Profit"].sum()
            ax.bar(df3.index,df3.values)
            ax.plot(df4.index,df4.values, color="red", marker=".")
            ax.set_xlabel(Field)
            ax.set_ylabel("Actual Profit compared to Target Profit")
            ax.legend(labels=[ 'Target Profit', 'Profit'])
           

    plt.show()

interactive(children=(Dropdown(description='style', options=('Solarize_Light2', '_classic_test_patch', 'bmh', …

This interactive visualization can be used by a user to get multiple insights.
The primary objective is to compare the Actual Sales values with the Target Sales value or the Actual Profit values with the Target Profit values. By comparing say for instance Sales with the Target Sales we can determine how much the actual sales have differed with the target sales.This can be now compared across multiple fields like :

Actual Sales with Target Sales values in various States.
Actual Sales with Target Sales values in various Market.
Actual Sales with Target Sales values for various Products.
Actual Sales with Target Sales values in various Product Line.
Actual Sales with Target Sales values for various Product Types.

Similarily the user can select Profit values to compare how the actual profit has increased or decreased with respect to the target Profit values. This can again be compared across multiple fields like :

Actual Profit with Target Profit values in various States.
Actual Profit with Target Profit values in various Market.
Actual Profit with Target Profit values for various Products.
Actual Profit with Target Profit values in various Product Line.
Actual Profit with Target Profit values for various Product Types.

Furthermore, I have given full flexibility to the user to choose a style using a drop down widget. The user can select any style to view the visualization as desired by him/her.



In [15]:
pivot_table2=pd.pivot_table(global_coffee, index="State", columns="Product Type",values="Sales",aggfunc='mean',dropna=True,fill_value=0)

In [16]:
pivot_table2

Product Type,Coffee,Espresso,Herbal Tea,Tea
State,,,,
California,317.666667,461.000000,285.611111,215.277778
Colorado,212.111111,169.000000,199.388889,148.722222
Connecticut,319.166667,132.666667,162.666667,100.916667
Florida,242.000000,220.500000,122.222222,89.583333
Illinois,378.833333,535.000000,212.111111,169.000000
Iowa,46.333333,51.916667,446.500000,545.333333
Louisiana,109.416667,134.000000,205.666667,0.000000
Massachusetts,495.000000,228.333333,119.500000,76.000000
Missouri,162.666667,101.750000,124.666667,77.944444


In [17]:
# 1. Scales --for Bar Plot
# I have referred to my Assignment 6.
x_sch2 = bqplot.OrdinalScale()
y_sch2 = bqplot.LinearScale()

# 2. Axis
x_axh2 = bqplot.Axis(scale=x_sch2,label="Product")
y_axh2 = bqplot.Axis(scale=y_sch2, orientation="vertical",label="Sales")

# 3. Marks -- using bar plot
bar_to_plot2=bqplot.pyplot.bar(x=[], y=[], scales={'x':x_sch2, 'y':y_sch2})

# 4. figure!
fig_barplot2 = bqplot.Figure(marks=[bar_to_plot2], axes=[x_axh2, y_axh2])
#fig_barplot

In [18]:
# step1 to add scales - colors, x & y
# Referred to Jill Naiman's class code and my assignment 6(homework)
col_sc2 = bqplot.ColorScale(scheme="Purple") 
x_sc2 = bqplot.OrdinalScale() 
y_sc2 = bqplot.OrdinalScale()

#step2. Axis -- color & for x/y
ax_col2 = bqplot.ColorAxis(scale=col_sc2, orientation='vertical', side='right')
ax_x2 = bqplot.Axis(scale=x_sc2,label="Product Type")
ax_y2= bqplot.Axis(scale=y_sc2, orientation='vertical',label="State")

# Step3. Using heatmap to plot
heat_map2 = bqplot.GridHeatMap(color=pivot_table2.values,row =pivot_table2.index,column =pivot_table2.columns,
                             scales={'color':col_sc2, 'row':y_sc2, 'column':x_sc2}, interactions={'click':'select'},
                        anchor_style={'fill':'yellow'},unselected_style = {'opacity': 2.0})

#step4 Defined a function for the bar plot
def in_barplot2(a,b):
    val=global_coffee[(global_coffee["State"]==a) & (global_coffee["Product Type"]==b)]
    val = val[val['Product'] != 0] 
    val=val.groupby("Product")["Sales"].sum()
    return val

# step5. building interactions
mySelectedLabel2 = ipywidgets.Label()
mySelectedLabel2.value = 'Mean selected point here'
# step6 interaction function -- this is what happens on selection:
def on_selected2(change):
    #print(change)
    if len(change['owner'].selected) == 1: # this will allow the user to select one bin
        i,j = change['owner'].selected[0] # the only selected bin
        v = pivot_table2.values[i,j] #passing pivot1.values
        mySelectedLabel2.value = 'Mean Sales in the State : ' + pivot_table2.index[i] + ' for the Product Type : ' + pivot_table2.columns[j] + ' = ' + str(v)
# step 7 for linking  interaction function to the mark:
        new_coffee=in_barplot2(pivot_table2.index[i],pivot_table2.columns[j])
    
        bar_to_plot2.x=new_coffee.index
        bar_to_plot2.y=new_coffee.values
heat_map2.observe(on_selected2, 'selected')

# step8 Finally, putting together in a figure
fig_heatmap2 = bqplot.Figure(marks=[heat_map2], axes=[ax_col2, ax_x2, ax_y2])
fig_barplot2 = bqplot.Figure(marks=[bar_to_plot2], axes=[x_axh2, y_axh2])

# step9 to display on a dashboard.
myDashboard2 = ipywidgets.VBox([mySelectedLabel2, ipywidgets.VBox([fig_heatmap2,fig_barplot2])])
myDashboard2


This is an interactive dashboard.Again starting with the grid heat map here the main variable of interest is the mean of sales for each Product Type across different States. This simply means that a user working on my dashboard will be able to visualize through this grid heat map what is the value of the mean of sales for different types of products like coffee, espresso, herbal tea and tea in various states.

When a user wants to find the sales of Coffee or tea in any state say for instance California then using this dashboard this can be achieved. Further we can compare these sales across each product type or compare the sales of a particular product type across the states.

Next to add another dimension to my visualization I have included a bar graph. This bar chart shows the total sales for each product when a cell from the heat map is selected i.e a product type in a  particular state is selected.

My dashboard is interactive. By selecting a cell one will get the mean value of sales of a product type beloging to a particular state. Furthermore, the bar chart then breaks the sales down to all products and shows the total sales of the product made in that state. For instance using the above dashboard, when the cell is selected for the mean of sales for tea in New York then the bar graph will display total sales for each product like Darjeeling tea, Earl Grey Tea or Green Tea belonging to product type (tea) in New York.
This can be summarised to identify what product type is popular in each state and additionally what products for a particular product type is sold in each state.